# Managing Exceptions with ResultTypes

In this post, we want to briefly review some examples of exception handling, present the [ResultTypes](https://github.com/iamed2/ResultTypes.jl) package as an alternative and finally, show how these mechanisms can cooperate nicely.

## Exceptions in Julia

Julia supports [Exceptions](https://docs.julialang.org/en/v1/manual/control-flow/#Exception-Handling-1) with mechanisms well-known from other languages:

A function can `throw` an exception when facing invalid input or a situation that is in some otherwise unexpected.
The caller can then _handle_ the exception by wrapping the function call in a `try`/`catch` block.
If not handled, the exception will travel upwards on the call stack all the way to the REPL (or even stop the process) and print a stack trace. Julia offers several built-in exception types, one can also define custom `Exception` types, or simply call the `error` function as a shortcut to throw an `ErrorException` with a given message.

In [1]:
using HTTP

In [2]:
function my_get(url)
    if !startswith(url, "https://")
        throw(DomainError("$url: only HTTPS requests allowed!"))
    end
    return HTTP.get(url)
end

my_get (generic function with 1 method)

In [3]:
my_get("http://httpbin.org")

DomainError: DomainError with http://httpbin.org: only HTTPS requests allowed!:


In [4]:
# Example: third-party function throws Exception
HTTP.get("http://httpbin.org/status/404")

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(404, HTTP.Messages.Response:
"""
HTTP/1.1 404 Not Found
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:10:53 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

In [5]:
function handled_get(url)
    try
        return my_get(url)
    catch e
        if isa(e, DomainError)
            return my_get(replace(url, "http" => "https", count=1))
        else
            rethrow(e)
        end
    end
end

handled_get (generic function with 1 method)

In [6]:
handled_get("http://httpbin.org/status/200")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:10:57 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

"""

In [7]:
handled_get("https://httpbin.org/status/404")

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(404, HTTP.Messages.Response:
"""
HTTP/1.1 404 Not Found
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:10:58 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

## ResultTypes

TODO: summarize [ResultTypes](https://github.com/iamed2/ResultTypes.jl), talk about performance benefits, but also hint at others (local error handling, explicit about possible problems).

In [8]:
using ResultTypes

In [9]:
# Example: repeat custom example above
function my_get2(url)::Result{HTTP.Messages.Response, DomainError}
    if !startswith(url, "https://")
        return DomainError(url, "Insecure protocol!")
    end
    return HTTP.get(url)
end

my_get2 (generic function with 1 method)

In [10]:
my_get2("https://httpbin.org/status/200")

Result(HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:10:59 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

In [11]:
my_get2("http://httpbin.org/status/200")

ErrorResult(HTTP.Messages.Response, DomainError("http://httpbin.org/status/200", "Insecure protocol!"))

In [12]:
my_get2("https://httpbin.org/status/404")

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(404, HTTP.Messages.Response:
"""
HTTP/1.1 404 Not Found
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:11:00 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

TODO: Discuss scope of `Result`: Can we only apply it _internally_ to our own packages? How to work together with other packages that don't use `Result`?
- Can not avoid that functions we call will throw exceptions (even in Base).
- Can not expect that code that uses our library will deal with `Result`

## Systematic Error-Handling in C++

TODO: Summarize (first part) of Andrei Alexandrescu's talk, with `Expected<T>` type; Compare to `Result`.
[video](https://channel9.msdn.com/Shows/Going+Deep/C-and-Beyond-2012-Andrei-Alexandrescu-Systematic-Error-Handling-in-C)
[slides](https://onedrive.live.com/?cid=F1B8FF18A2AEC5C5&id=F1B8FF18A2AEC5C5%211158&parId=root&o=OneUp)

Focus on slide 27 (_Icing_) with `fromCode`:

```
template <class F>
static Expected fromCode(F fun) {
    try {
        returnExpected(fun());
    } catch(...) {
        return fromException();
    }
}

auto r = Expected<string>::fromCode([] {...});
```

This allows converting capturing exception-throwing code and turn it into `Result` return code!
Even applies to third-party code, can connect the two worlds in both ways!

## Wrapping Callees

In [13]:
# Example for wrapping called code
function my_get3(url)
    if !startswith(url, "https://")
        return ErrorResult(DomainError(url, "Insecure protocol!"))
    end
    try
        return Result(HTTP.get(url))
    catch e
        return ErrorResult(e)
    end
end 

my_get3 (generic function with 1 method)

In [14]:
my_get3("https://httpbin.org/status/200")

Result(HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:11:00 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

In [15]:
my_get3("http://httpbin.org/status/200")

ErrorResult(Any, DomainError("http://httpbin.org/status/200", "Insecure protocol!"))

In [16]:
my_get3("https://httpbin.org/status/400")

ErrorResult(Any, HTTP.ExceptionRequest.StatusError(400, HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:11:01 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

"""))

In [17]:
?ResultTypes.iserror

```
ResultTypes.iserror(x) -> Bool
```

If `x` is an `Exception`, return `true`. If `x` is an `ErrorResult` (a `Result` containing an `Exception`), return `true`. Return `false` in all other cases.


# Wrapping for Callers

In [18]:
# Example for throwing from `Result` results
function my_get4(url)
    result = my_get3(url)
    if ResultTypes.iserror(result)
        throw(unwrap_error(result))
    end
    return unwrap(result)
end

my_get4 (generic function with 1 method)

In [19]:
my_get4("https://httpbin.org/status/200")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:11:03 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

"""

In [20]:
my_get4("http://httpbin.org/status/200")

DomainError: DomainError with http://httpbin.org/status/200:
Insecure protocol!

In [21]:
my_get4("https://httpbin.org/status/500")

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(500, HTTP.Messages.Response:
"""
HTTP/1.1 500 Internal Server Error
Connection: keep-alive
Server: gunicorn/19.9.0
Date: Fri, 25 Jan 2019 21:11:15 GMT
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Content-Length: 0
Via: 1.1 vegur

""")

Can we get the complete stack trace in the last case, just like with `rethrow` before?